# Exercise 1
---

## Problem: 
1. Ask user to supply the mean and standard deviation of a 1D Gaussian
2. For the given mean and std, draw 1000 random samples from $[-2\sigma,+\sigma]$ to build a datset $D$
3. Split the dataset into train ($90\%$) and test ($10\%$): $D = \{ D_{train},D_{test} \}$
4. Using pure **Python3** and **Nump**, build a $3$-layer neural network:
$$\mathrm{Layers:} \{  1 - 64 - ReLU - 64 - ReLU - 64 - 1 - sigmoid \}$$
5. Initialize the weights using  a Gaussian distribution with zero mean and std of $1$
6. Train for 20 epochs and evaluate performance of network

## Solution
---

In [15]:
from utils import sigmoid, relu, slice_dataset
import numpy as np  
import matplotlib.pyplot as plt

In [16]:
# mu = float(input("Enter mean:"))
# print('Mean:', mu)

# sigma = float(input("Enter standard deviation:"))
# print('Standard Deviation:', sigma)

mu = 0.0
sigma = 0.1

# make normal distribution for user input parameters
dataset = np.random.normal(mu, sigma, 10)

# count, bins, ignored = plt.hist(dataset, 30, density=True)
# plt.plot(bins, 1/(sigma * np.sqrt(2 * np.pi)) *
#                np.exp( - (bins - mu)**2 / (2 * sigma**2) ),
#          linewidth=2, color='r')
# plt.show()

print(slice_dataset(dataset, -2*sigma, 2*sigma))

0
